# The Genesis process

Creating a new game requires the creation of a system and planets, factions and people.

This notebook tests that process as well as documents it. 

In [1]:
import numpy as np
import pandas as pd

import sys, os, yaml, ssl, asyncio
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")

%load_ext lab_black

In [2]:
from app import creators
from app import objects

In [3]:
from app.creators import homeworld
from app.creators import universe

An example `data` set for testing. This is the user form that the user submits when creating a new game. 

You can get this from the template at `app\templates\app\creation\genesis.js`


In [4]:
data = {
    "label": "form",
    "name": "worldgenform",
    "objid": "0000000000001",
    "owner": "user.username",
    "username": "user.username",
    "accountid": "0000000000001",
    "conformity": 0.5,
    "constitution": 0.5,
    "literacy": 0.5,
    "aggression": 0.5,
    "num_planets": 4,
    "num_moons": 10,
    "starting_pop": 7,
    "organics": 0.5,
    "minerals": 0.5,
}

You'll also need to configuration files.

In [5]:
conf = creators.universe.configurations.get_configurations()

Each object as an `__init__` function that creates it, however not all data is spawed on init. 

## The Solar System and Planets

In [6]:
system = universe.celestials.System(data)
system

<sytem: ordered; 0281868200696; Rocroutykon>

In [7]:
system.get_data()

{'objid': '0281868200696',
 'name': 'Rocroutykon',
 'label': 'sytem',
 'class': 'ordered',
 'isHomeSystem': True,
 'glat': -36.562,
 'glon': 21.217,
 'gelat': -7.063}

Each object as an `__init__` function that creates it, and populates it with data available at that time. Celestial objects have dependancies on other objects. 

In [8]:
star = objects.celestials.Star(conf["star_config"], system)
star.get_data()

{'name': 'U',
 'class': 'G',
 'objid': '5628568873816',
 'label': 'star',
 'radius': 106}

Each object also inherits a `get_fundamentals()` method that ensures that needed default values are present. `get_data()` extends that functionality. This ensures that objects can alwasy interact with the graph. 

In [9]:
star.get_fundimentals()

{'name': 'U', 'class': 'G', 'objid': '5628568873816', 'label': 'star'}

Procedurally generated planets chose from a list of potential types in the `conf`. To force a particular kind of outcome, reduce the options in the configuration. 

In [10]:
terrestrial_config = {"terrestrial": conf["planet_config"]["terrestrial"]}
home_planet = objects.celestials.Planet(conf=terrestrial_config, orbiting=star)
home_planet.get_data()

{'name': 'Silfuer',
 'class': 'terrestrial',
 'objid': '7851490176822',
 'label': 'planet',
 'radius': 0.741,
 'mass': 1.526,
 'orbitsDistance': 1.066,
 'orbitsId': '5628568873816',
 'orbitsName': 'U',
 'isSupportsLife': False,
 'isPopulated': False}

Creating a group of planets using list comprehension. Note that celestial objects have a custom `__repr__` function that makes them easy to manage.

In [11]:
planets = [
    objects.celestials.Planet(conf=conf["planet_config"], orbiting=star)
    for p in range(int(data["num_planets"]) - 1)
]
planets

[<planet: ice; 2703009727471; Seky>,
 <planet: dwarf; 4549945741951; Gnareimeshu>,
 <planet: gas; 4117501559459; Gasouthval>]

In [12]:
moons = [
    objects.celestials.Moon(conf["moon_config"], planets)
    for p in range(int(data["num_moons"]))
]
moons

[<moon: rocky; 6215648128275; Flo>,
 <moon: rocky; 9124857619792; Aujonay>,
 <moon: rocky; 3467772928058; Bangsunhia>,
 <moon: rocky; 7829507805293; Lo>,
 <moon: rocky; 1666179775401; Hatgalha>,
 <moon: rocky; 6371143704833; Ky>,
 <moon: rocky; 4360046358536; Bartle>,
 <moon: ice; 1631635978925; Puerye>,
 <moon: rocky; 1142901433778; Kasortradan>,
 <moon: rocky; 5565474300605; Haudospethau>]

Getting the nodes and edges of each by calling the `self.orbiting` propperty.

In [13]:
moons[0].orbiting

<planet: dwarf; 4549945741951; Gnareimeshu>

In [14]:
home_planet.orbiting

<star: G; 5628568873816; U>

Additionally, you can quickly navigate the system by referencing other objects. 

In [15]:
home_planet.orbiting.system

<sytem: ordered; 0281868200696; Rocroutykon>

Getting the nodes for the graph. Sandwich all of the items together and get the data using the same generic function.

In [16]:
all_entities = [system] + [star] + moons + planets + [home_planet]
all_nodes = [b.get_data() for b in all_entities]
pd.DataFrame(all_nodes)

,objid,name,label,class,isHomeSystem,glat,glon,gelat,radius,orbitsId,orbitsName,orbitsDistance,mass,isSupportsLife,isPopulated
0,0281868200696,Rocroutykon,sytem,ordered,True,-36.562,21.217,-7.063,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5628568873816,U,star,G,NaN,NaN,NaN,NaN,106.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,6215648128275,Flo,moon,rocky,NaN,NaN,NaN,NaN,0.007395,4549945741951,Gnareimeshu,0.1390,0.000330,False,False
3,9124857619792,Aujonay,moon,rocky,NaN,NaN,NaN,NaN,0.016248,4549945741951,Gnareimeshu,0.2850,0.000318,False,False
4,3467772928058,Bangsunhia,moon,rocky,NaN,NaN,NaN,NaN,0.945442,4117501559459,Gasouthval,4.0001,0.000142,False,False
5,7829507805293,Lo,moon,rocky,NaN,NaN,NaN,NaN,0.506076,4117501559459,Gasouthval,4.0001,0.000363,False,False
6,1666179775401,Hatgalha,moon,rocky,NaN,NaN,NaN,NaN,0.596887,2703009727471,Seky,3.7701,0.000231,False,False
7,6371143704833,Ky,moon,rocky,NaN,NaN,NaN,NaN,0.388076,4117501559459,Gasouthval,4.1700,0.000556,False,False
8,4360046358536,Bartle,moon,rocky,NaN,NaN,NaN,NaN,0.774933,4117501559459,Gasouthval,4.0001,0.000362,False,False
9,1631635978925,Puerye,moon,ice,NaN,NaN,NaN,NaN,0.144790,2703009727471,Seky,3.9690,0.004142,False,False


Getting the edge values to update the graph: 

In [17]:
orbiting_bodies = [home_planet] + planets + moons

orbiting_edges = [i.get_orbits_edge() for i in orbiting_bodies]

pd.DataFrame(orbiting_edges)

,node1,node2,label,orbit_distance
0,7851490176822,5628568873816,orbits,1.0660
1,2703009727471,5628568873816,orbits,25.1600
2,4549945741951,5628568873816,orbits,47.6530
3,4117501559459,5628568873816,orbits,9.2930
4,6215648128275,4549945741951,orbits,0.0100
5,9124857619792,4549945741951,orbits,0.1560
6,3467772928058,4117501559459,orbits,0.0001
7,7829507805293,4117501559459,orbits,0.0001
8,1666179775401,2703009727471,orbits,0.0001
9,6371143704833,4117501559459,orbits,0.1700


Check the orbiting logic

In [18]:
[
    f"{i.type} {i.label} orbits a {i.orbiting.type} {i.orbiting.label}"
    for i in orbiting_bodies
]

['terrestrial planet orbits a G star',
 'ice planet orbits a G star',
 'dwarf planet orbits a G star',
 'gas planet orbits a G star',
 'rocky moon orbits a dwarf planet',
 'rocky moon orbits a dwarf planet',
 'rocky moon orbits a gas planet',
 'rocky moon orbits a gas planet',
 'rocky moon orbits a ice planet',
 'rocky moon orbits a gas planet',
 'rocky moon orbits a gas planet',
 'ice moon orbits a ice planet',
 'rocky moon orbits a ice planet',
 'rocky moon orbits a ice planet']

In [19]:
system_bodies = orbiting_bodies + [star]

system_edges = [i.get_in_system_edge() for i in system_bodies]

pd.DataFrame(system_edges)

,node1,node2,label
0,7851490176822,0281868200696,isIn
1,2703009727471,0281868200696,isIn
2,4549945741951,0281868200696,isIn
3,4117501559459,0281868200696,isIn
4,6215648128275,0281868200696,isIn
5,9124857619792,0281868200696,isIn
6,3467772928058,0281868200696,isIn
7,7829507805293,0281868200696,isIn
8,1666179775401,0281868200696,isIn
9,6371143704833,0281868200696,isIn


### Scanning the homeworld
the homeworld already has some resources known. `scan_body()` is inherited by the base object. So everyone should be able to do it. 

In [20]:
home_planet.scan_body()
home_planet.resources

[<resource: 2065117730997; Organic>,
 <resource: 2997330210350; Common Minerals>,
 <resource: 6328151976488; Rare Minerals>,
 <resource: 3425239038004; Water>]

In [21]:
pd.DataFrame([i.get_data() for i in home_planet.resources])

,name,objid,label,volume,max_volume,description,replenish_rate
0,Organic,2065117730997,resource,968,968,bilogical material that can be consumed by pops,10.0
1,Common Minerals,2997330210350,resource,104,104,Iron and other common material used in constru...,NaN
2,Rare Minerals,6328151976488,resource,57,57,"lithium, silver and other rare minerals used i...",NaN
3,Water,3425239038004,resource,9782,9782,"H2O ready to be consumed, either frozen or in ...",NaN


In [22]:
pd.DataFrame([i.get_location_edge() for i in home_planet.resources])

,node1,node2,label
0,7851490176822,2065117730997,has
1,7851490176822,2997330210350,has
2,7851490176822,6328151976488,has
3,7851490176822,3425239038004,has


In [23]:
[
    f"{i.location.type} {i.location.label} has {i.volume} {i.name}"
    for i in home_planet.resources
]

['terrestrial planet has 968 Organic',
 'terrestrial planet has 104 Common Minerals',
 'terrestrial planet has 57 Rare Minerals',
 'terrestrial planet has 9782 Water']

# The Full Automated Process

Usefull as QA, to ensure that the process will run end-to-end

In [24]:
homesystem_data = universe.build_homeSystem(data, username="notebook")
pd.DataFrame(homesystem_data["nodes"])

,objid,name,label,class,isHomeSystem,glat,glon,gelat,radius,orbitsId,...,accountid,conformity,constitution,literacy,aggression,num_planets,num_moons,starting_pop,organics,minerals
0,0192108546364,Ren,sytem,ordered,True,16.589,35.765,7.588,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4509610436294,Shitla,star,G,NaN,NaN,NaN,NaN,106.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1286293973616,Haidalbarno,moon,ice,NaN,NaN,NaN,NaN,0.228991,7746724942114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6561362026016,Hou,moon,ice,NaN,NaN,NaN,NaN,0.066985,7746724942114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4810399787690,Sarhenty,moon,ice,NaN,NaN,NaN,NaN,0.022010,6246550689367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4375862359302,Rintou,moon,terrestrial,NaN,NaN,NaN,NaN,0.061255,6246550689367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6480361674208,Osnan,moon,ice,NaN,NaN,NaN,NaN,0.180598,7746724942114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,9240314238333,Ganmelhigen,moon,ice,NaN,NaN,NaN,NaN,0.006150,3306417895379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2199502327334,Copat,moon,rocky,NaN,NaN,NaN,NaN,0.160780,6246550689367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9422434206024,Yesedou,moon,rocky,NaN,NaN,NaN,NaN,0.536967,7746724942114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
print(
    f"{len(homesystem_data['edges'])}  total edges for {len(homesystem_data['nodes'])} objects"
)
pd.DataFrame(homesystem_data["edges"]).groupby("label").count()

37  total edges for 22 objects


,node1,node2,orbit_distance
label,,,
created_from_form,1,1,0
has,4,4,0
isInSystem,16,16,0
orbits,15,15,15
submitted,1,1,0


# The planet Surface

In [ ]:
homeworld_data = homeworld.build_people(data)
homeworld_data

([{'name': 'Badla',
   'objid': '2152333961637',
   'label': 'species',
   'consumes': ['Organic'],
   'effuses': ['Organic waste', 'Plastics'],
   'viral_resilience': 0.7,
   'habitat_resilience': 0.2},
  {'name': 'Fucas Hyjiangfuca',
   'objid': '6714538532858',
   'label': 'pop',
   'conformity': 0.574,
   'literacy': 0.277,
   'aggression': 0.487,
   'constitution': 0.577,
   'health': 0.5,
   'isInFaction': '8424616458051',
   'industry': 0.532,
   'wealth': 0.4045,
   'factionLoyalty': 0.655,
   'isIdle': 'true'},
  {'name': 'Rosar Orbarnu',
   'objid': '8368216969422',
   'label': 'pop',
   'conformity': 0.606,
   'literacy': 0.605,
   'aggression': 0.553,
   'constitution': 0.547,
   'health': 0.5,
   'isInFaction': '8422192744305',
   'industry': 0.55,
   'wealth': 0.5775,
   'factionLoyalty': 0.778,
   'isIdle': 'true'},
  {'name': 'Rosar Cor',
   'objid': '2956170443012',
   'label': 'pop',
   'conformity': 0.604,
   'literacy': 0.526,
   'aggression': 0.597,
   'constitutio